# Dynamisches Labyrinth

## TODOS

1.1 CSV einlesen

In [161]:
import csv

# Pfad zur CSV-Datei
csv_datei_pfad = './data/Puzzle_easy_3.csv'

# Liste zum Speichern der Daten
spielfeld = []

# CSV-Datei öffnen und Daten einlesen
with open(csv_datei_pfad, 'r') as csv_datei:
    csv_reader = csv.reader(csv_datei, delimiter=";")
    
    # Zeile für Zeile durch die CSV-Datei iterieren
    for zeile in csv_reader:
        spielfeld.append(zeile)

# Die 'daten'-Liste enthält nun die Daten aus der CSV-Datei
# daten[1] <- Spalte 1 mit Anführungszeichen
# daten[1][0] <- Spalte 1, zweites Array immer mit Index 0
# daten[1][0][2] <- Spalte 1, mit Index 2 gibt eine einzelene Zahl/Spielfeld
# print(spielfeld[2][0][0])

1.2 Grid definieren und initalisieren

In [162]:
# überarbeitet
class Grid:
    def __init__(self, src, dst, matrix):
        self.src = src
        self.dst = dst
        self.rows = len(matrix)-1
        self.cols = len(spielfeld[0]) if self.rows > 0 else 0 #self.cols = len(spielfeld[0][0].split(";")) if self.rows > 0 else 0
        self.nodes = [(i, j) for i in range(self.rows) for j in range(self.cols)]
        self.grid_values = [[0 for _ in range(self.cols)] for _ in range(self.rows)]

        for i in range(self.rows):
            for j in range(self.cols):
                self.grid_values[i][j] = int(matrix[i][j])

    def get_value(self, row, col):
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid_values[row][col]
        return None  # Wert außerhalb der Grenzen des Grids

    def get_nodes(self):
        return self.nodes
    
    def get_adjacent(self, node):
        x, y = node
        adjacent_nodes = []

        # Horizontale Nachbarn
        for dx in [-1, 1]:
            new_x, new_y = x + dx, y
            if 0 <= new_x < self.rows and 0 <= new_y < self.cols:
                adjacent_nodes.append((new_x, new_y))

        # Vertikale Nachbarn
        for dy in [-1, 1]:
            new_x, new_y = x, y + dy
            if 0 <= new_x < self.rows and 0 <= new_y < self.cols:
                adjacent_nodes.append((new_x, new_y))

        return adjacent_nodes

spielfeld_grid = Grid((0,0), (5,4), spielfeld)

# Grid 
print("Nodes: ", spielfeld_grid.get_nodes())
test_node = (1,1)
print("Nachbarn: ", spielfeld_grid.get_adjacent(test_node))
print("Anzahl Nachbarn", len(spielfeld_grid.get_adjacent(test_node)))
print("Wert Node", spielfeld_grid.grid_values)
print("Wert Node", spielfeld_grid.get_value(0,1))

Nodes:  [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3)]
Nachbarn:  [(0, 1), (2, 1), (1, 0), (1, 2)]
Anzahl Nachbarn 4
Wert Node [[2, 8, 0, 7], [1, 5, 8, 1], [0, 7, 7, 1], [6, 5, 5, 5], [4, 4, 6, 6]]
Wert Node 8


## 2. Spielsteine definieren

Erklärung: Es gibt 10 verschiedene Spielsteine die angeben, in welche Richtung der Spieler sich bewegen und nicht bewegen kann. Dabei werden die verschiedenen Spielsteine in einer
- Matrix von 0 - 9 dargestellt und
- jeder Spielstein erhält zudem eine 2x2 Matrix der die möglichen Richtungen angibt. 

Dabei gilt folgendes: [ left, top, right, bottom] wobei eine 1 für eine Verbindung und eine 0 für keine Verbindung steht zum nächsten Feld steht. Als Beipsiel, [0, 0, 1, 1] steht für [-, -, right, bottom], also ist dieser Spielstein eine Ecke die eine Verbindung von unten nach rechts besitzt.

In [163]:
# [ left, top, right, bottom]
tile0 = [0, 0, 1, 1]
tile1 = [1, 0, 0, 1]
tile2 = [0, 1, 1, 0]
tile3 = [1, 1, 0, 0]
tile4 = [1, 0, 1, 1]
tile5 = [1, 1, 0, 1]
tile6 = [1, 1, 1, 0]
tile7 = [0, 1, 1, 1]
tile8 = [0, 1, 0, 1]
tile9 = [1, 0, 1, 0]

In [164]:
# Kodierung für das Spielfeld
tiles = [tile0, tile1, tile2, tile3, tile4, tile5, tile6, tile8, tile9]

# Beispiel
# Spielfield hat bei 0;0 die Zahl 0, dadurch ergibt sich tiles[0] => [0, 0, 1, 1]
#print(tiles[0])#
print("Wert Spielstein", spielfeld_grid.get_value(0,1))
print("Spielstein Kodierung: ", tiles[spielfeld_grid.get_value(0,1)])

Wert Spielstein 8
Spielstein Kodierung:  [1, 0, 1, 0]


## 3. A* anhand lösbarem Beispiel

Some text

In [165]:
# Replace this with your actual heuristic function
def heuristic_cost_estimate(current, goal):
    # Example: Euclidean distance heuristic
    x1, y1 = current
    x2, y2 = goal
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

In [166]:
def a_star(grid):
    nodes = grid.get_nodes()
    closed_set = []
    open_set = [grid.src]
    prev = {node: None for node in nodes}
    g_score = {node: float("inf") for node in nodes}
    src = grid.src
    dst = grid.dst
    g_score[src] = 0
    f_score = {node: float("inf") for node in nodes}
    f_score[src] = heuristic_cost_estimate(src, dst)
    
    while open_set:
        u = min(open_set, key=f_score.get)
        open_set.remove(u)
        closed_set.append(u)
        
        if u == dst:
            print('Found end')
            break
        
        for v in grid.get_adjacent(u):
            new_g = g_score[u] + 1
            
            if v not in open_set and v not in closed_set:
                open_set.append(v)
                g_score[v] = new_g
                f_score[v] = new_g + heuristic_cost_estimate(v, dst)
                prev[v] = u
            elif new_g < g_score[v]:
                g_score[v] = new_g
                f_score[v] = new_g + heuristic_cost_estimate(v, dst)
                prev[v] = u
                
                if v in closed_set:
                    open_set.append(v)
                    closed_set.remove(v)
    
    # Now reconstruct the path if it was found
    path = []
    if prev[dst] is not None:
        current = dst
        while current is not None:
            path.append(current)
            current = prev[current]
        path = path[::-1]  # Reverse to get the path from src to dst
    
    return path


In [167]:
a_star(spielfeld_grid)

KeyError: (5, 4)

## Grafische Ausgabe

In [ ]:
matrix = [
    [2, 8, 0, 5],
    [1, 5, 8, 1],
    [0, 7, 7, 1],
    [6, 5, 5, 5],
    [4, 4, 6, 6]
]

In [ ]:
string_map = {
    0: "┌",
    1: "┐",
    2: "└",
    3: "┘",
    4: "┬",
    5: "┤",
    6: "┴",
    7: "├",
    8: "|",
    9: "-",
}

In [ ]:
# Create a matrix of strings
string_matrix = [[string_map[num] for num in row] for row in matrix]

# Display the string matrix
for row in string_matrix:
    print(" ".join(row))

└ | ┌ ┤
┐ ┤ | ┐
┌ ├ ├ ┐
┴ ┤ ┤ ┤
┬ ┬ ┴ ┴


3. A* anhand lösbarem Beispiel

- checken grenzen
    - wenn laufen nicht möglich, dann schieben
        
- verschieben  
    - Zeile über Spielfigur in möglich Richtung
    
- checken funktion/algo
- Listen führen: Open/Close

- Heuristik:
    - 1 Schritt = +1
    - 1 Verschieben = +1
    - Heuristischer wert = Schritte bis zum Ziel
        -   ABER wenn nicht Öffnung nach oben +2

WAS TUN BEI GLEICHSTAND?
AB WANN VERSCHIEBEN (auch wenn laufen möglich)

- jeden möglichen weg berechnen
- jeden möglichen weg nach verschiebungen